<a href="https://colab.research.google.com/github/vasiliyeskin/MachineLearningExperiences/blob/master/computer_vision/object_detection/object_detection_api_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Пример поиска и распознавания объектов на изображении**

Используется TensorFlow Object Detection API. Инструкции по установке:

Linux.
Windows.
Видео на YouTube с демонстрацией распознавания объектов.


**Imports and Setup of Object Detection**

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

**Сам код**

In [3]:
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
%matplotlib inline

# **Загружаем модель TensorFlow**
Модель можно загрузить из Tensorflow detection model zoo. Из каталога с моделью нужен всего один файл - frozen_inference_graph.pb. Это "замороженная" предварительно обученная сеть, которую можно использовать для распознавания (inference).

Есть два типа моделей, отличаются выходными данными, указано в слобце Outputs в таблице:

    Boxes - модель выдает прямоугольник, внутри которого находится найденный объект
    Masks - модель выдает маску пикселей, которые соответствуют объекту

In [11]:
# download model with mask
# !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz -O mask_rcnn_inception_resnet_v2.tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz -O faster_rcnn_inception_resnet_v2.tar.gz

--2020-08-06 21:13:01--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.20.128, 2607:f8b0:400e:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.20.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 447961769 (427M) [application/x-tar]
Saving to: ‘faster_rcnn_inception_resnet_v2.tar.gz’

faster_rcnn_incepti 100%[===================>] 427.21M   363MB/s    in 1.2s    

2020-08-06 21:13:02 (363 MB/s) - ‘faster_rcnn_inception_resnet_v2.tar.gz’ saved [447961769/447961769]



In [ ]:
!ls
# !tar -xvf mask_rcnn_inception_resnet_v2.tar.gz
!tar -xvf faster_rcnn_inception_resnet_v2.tar.gz

In [14]:
# Тип выходных данных модели - Boxes
model_file_name = 'faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/saved_model/saved_model.pb'
# Тип выходных данных модели - Masks
# model_file_name = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/saved_model.pb'

In [18]:
# Загружаем предварительно обученную модель в память
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.compat.v1.gfile.GFile(model_file_name, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

DecodeError: ignored